In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import norm
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression, RFE
import pickle
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [8]:
def scalerX(indep_X):
    sc = StandardScaler()
    indep_X = sc.fit_transform(indep_X)
    return indep_X

def scalerY(dep_Y):
    scy=StandardScaler()
    dep_Y = scy.fit_transform(dep_Y)
    return dep_Y

def r2_prediction(grid,indep_X,dep_Y):
    y_pred = grid.predict(indep_X)
    r2=r2_score(dep_Y,y_pred)
    return r2

def Linear(indep_X,dep_Y):
    param_grid={'fit_intercept':[True,False],'copy_X':[True,False],'positive':[False,True]}
    grid=GridSearchCV(LinearRegression(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2

def svm(indep_X,dep_Y):
    param_grid={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'C':[10,100],'gamma':['auto','scale']}
    grid=GridSearchCV(SVR(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2 

def Decision(indep_X,dep_Y):
    param_grid={'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],'splitter':['best', 'random'],'max_features':['sqrt', 'log2']}
    grid=GridSearchCV(DecisionTreeRegressor(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2 

def random(indep_X,dep_Y):
    param_grid={'n_estimators':[10,50,100],'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],'max_features':['sqrt', 'log2']}
    grid=GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2  

def rfeFeature(indep_X,dep_Y,n):
    rfelist=[]
        
    lin = LinearRegression()
    SVRl = SVR(kernel = 'linear')
    dec = DecisionTreeRegressor(random_state = 0)
    rf = RandomForestRegressor(n_estimators = 10, random_state = 0)

    rfemodellist=[lin,SVRl,dec,rf] 
    for i in   rfemodellist:
        print(i)
        rfe=RFE(i,n_features_to_select=n, step=1)
        fit1=rfe.fit(indep_X,dep_Y)
        rfe_features=fit1.transform(indep_X)
        selectedCol=indep_X.columns[fit1.get_support(indices=True)]
        rfelist.append(rfe_features)
    return rfelist,selectedCol

def rfe_regression(acclog,accsvml,accdes,accrf): 
    rfedataframe=pd.DataFrame(index=['Linear','SVC','Random','DecisionTree'],columns=['Linear','SVM','Decision','Random'])

    for number,idex in enumerate(rfedataframe.index):
        
        rfedataframe['Linear'][idex]=acclog[number]       
        rfedataframe['SVM'][idex]=accsvm[number]
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe

In [9]:
dataset=pd.read_csv('sales_prep.csv',index_col=None)
df=dataset

dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

indep_X=dataset.drop('Sales_Revenue',axis=1)
dep_Y=dataset[['Sales_Revenue']]

In [10]:
rfelist,selected=rfeFeature(indep_X,dep_Y,3) 

acclin=[]
accsvm=[]
accdes=[]
accrf=[]


for i in rfelist:   
    indep_X=scalerX(i)
    dep_Y=scalerY(dep_Y) 
    
    r2_lin=Linear(indep_X,dep_Y)
    acclin.append(r2_lin)
    
    r2_sl=svm(indep_X,dep_Y)    
    accsvm.append(r2_sl)
    
    r2_d=Decision(indep_X,dep_Y)
    accdes.append(r2_d)
    
    r2_r=random(indep_X,dep_Y)
    accrf.append(r2_r)
    
    
result=rfe_regression(acclin,accsvm,accdes,accrf)


LinearRegression()
SVR(kernel='linear')


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=10, random_state=0)


C:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimato

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\tree\_classes.py", line 1377, in fit
    super()._fit(
  File "C:\Anaconda\Lib\site-packages\sklearn\tree\_class

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
30 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\ensemble\_forest.py", line 409, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which 

In [5]:
#2
result

,Linear,SVM,Decision,Random
Linear,0.009285,0.004343,0.010468,0.009206
SVC,0.0,-0.002473,1.0,0.824772
Random,0.0,0.002614,1.0,0.821489
DecisionTree,0.00374,-0.001585,0.998382,0.82386


In [11]:
#3
result

,Linear,SVM,Decision,Random
Linear,0.009285,0.008159,0.01753,0.016974
SVC,0.008424,0.003308,1.0,0.851784
Random,0.0,0.001388,1.0,0.820429
DecisionTree,0.008993,0.003553,1.0,0.842473
